# POC Projeto de Dissertação

## Sistema para checagem de doações de campanhas eleitorais por pessoas físicas e suas relações com empresas

**Objetivo**

Essa POC (Proof of Concept) tem o objetivo de validar os datasets, APIs, ferramentas e ideias a serem utilizadas no projeto através de um escopo menor.

**Candidato escolhido**: Geraldo Alckmin (PSDB)

**Motivação**: é o pré-candidato a presidente mais consolidado (com chances de concorrer) entre os que se apresentaram até agora. PSDB sempre apresenta candidatos à presidência, governo dos estados e capitais, o que nos dá uma volume maior de dados de doações pra trabalhar, além de podemos usar dados de apoio de campanhas anteriores do Alckmin para prefeitura/governo de SP.

**Eleições do Alckmin**

- 2002 - Governo do estado de São Paulo (eleito no segundo turno)
- 2006 - Presidência do Brasil (Derrotado no segundo turno)
- 2008 - Prefeitura de São Paulo (Derrotado no primeiro turno)
- 2010 - Governo do estado de São Paulo (eleito no primeiro turno)
- 2014 - Governo do estado de São Paulo (eleito no primeiro turno)

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text

doacoes_candidato = pd.read_csv('/home/aiquis/Datasets/prestacao_contas_2014_sp/receitas_candidatos_2014_SP.txt',
                                sep=';', encoding='latin_1',
                                dtype={'CNPJ Prestador Conta': str, 'CPF do candidato': str,
                                       'CPF/CNPJ do doador': str, 'CPF/CNPJ do doador originário': str},
                                thousands='.', decimal=',',
                                low_memory=True, na_values=['#NULO', -1])

doacoes_candidato.columns = ['cod_eleicao', 'desc_eleicao', 'data_hora',
                             'cnpj_prestador_conta', 'sequencial_candidato',
                             'uf', 'sigla_partido', 'num_candidato', 'cargo',
                             'nome_candidato', 'cpf_candidato', 'num_recibo_eleitoral',
                             'num_documento', 'cpf_cnpj_doador', 'nome_doador',
                             'nome_doador_receita', 'sigla_ue_doador', 'num_partido_doador',
                             'num_candidato_doador', 'cod_setor_econ_doador',
                             'setor_econ_doador', 'data_receita', 'valor_receita',
                             'tipo_receita', 'fonte_recurso', 'especie_recurso',
                             'desc_receita', 'cpf_cnpj_doador_originario',
                             'nome_doador_originario', 'tipo_doador_originario',
                             'setor_econ_doador_originario', 'nome_doador_originario_rf']

doacoes_alckmin = doacoes_candidato[doacoes_candidato.cpf_candidato == 54914906872]

doacoes_alckmin = doacoes_candidato[['cpf_cnpj_doador', 'nome_doador',
                                     'nome_doador_receita', 'sigla_ue_doador', 'cod_setor_econ_doador', 'setor_econ_doador', 'valor_receita',
                                     'tipo_receita', 'tipo_receita', 'fonte_recurso', 'especie_recurso',
                                     'desc_receita', 'cpf_cnpj_doador_originario',
                                     'nome_doador_originario', 'tipo_doador_originario',
                                     'setor_econ_doador_originario', 'nome_doador_originario_rf']]

doacoes_alckmin.info()  # 80.876 registros

# Separando os diferentes tipos de doadores em DataFrames diferentes para analisá-los separadamente


doacoes_alckmin_pf = doacoes_alckmin[doacoes_alckmin.tipo_doador_originario == 'F']
# 3.089 registros
doacoes_alckmin_pj = doacoes_alckmin[doacoes_alckmin.tipo_doador_originario == 'J']
# 14.470 registros
doacoes_alckmin_nan = doacoes_alckmin[doacoes_alckmin.tipo_doador_originario.isnull(
)]  # 63.317 registros

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80876 entries, 0 to 80875
Data columns (total 17 columns):
cpf_cnpj_doador                 80747 non-null object
nome_doador                     80747 non-null object
nome_doador_receita             80736 non-null object
sigla_ue_doador                 46316 non-null object
cod_setor_econ_doador           48511 non-null float64
setor_econ_doador               48511 non-null object
valor_receita                   80876 non-null float64
tipo_receita                    80876 non-null object
tipo_receita                    80876 non-null object
fonte_recurso                   80876 non-null object
especie_recurso                 80876 non-null object
desc_receita                    53687 non-null object
cpf_cnpj_doador_originario      17559 non-null object
nome_doador_originario          17565 non-null object
tipo_doador_originario          17559 non-null object
setor_econ_doador_originario    14447 non-null object
nome_doador_originario_rf

O volume de doações feitos pelas próprias instituições partidárias é muito alto e acaba ficando descrepante com as doações feitas por empresas ou pessoas. Para ter uma análise melhor, vou retirar das doações as doações que tiveram `nome_doador` que contenham os seguintes termos:

- Direção
- Comitê
- ELEIÇÃO 2014
- ELEIÇAO 2014
- ELEICAO 2014

In [17]:
def sum_doacoes_cpfcnpj(doacoes):
    doacoes_agrupadas = doacoes.groupby(['cpf_cnpj_doador', 'nome_doador_receita'])[
        'valor_receita'].sum().to_frame(name='sum').sort_values('sum', ascending=False)
    return doacoes_agrupadas


def limpa_doacoes_partido(doacoes):
    doacoes_limpas = (~doacoes['nome_doador_receita'].str.contains('Direção', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('Comitê', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('ELEIÇÃO', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('ELEIÇAO', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('ELEICAO', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('ELEIÇÕES', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('ELEIÇOES', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('ELEICOES', na=False) &
                      ~doacoes['nome_doador_receita'].str.contains('PARTIDO', na=False))
    doacoes = doacoes[doacoes_limpas == True]
    return doacoes


doacoes_alckmin_nan = limpa_doacoes_partido(doacoes_alckmin_nan)
doacoes_alckmin_nan = sum_doacoes_cpfcnpj(doacoes_alckmin_nan)

doacoes_alckmin_nan.index.rename('cpf_cnpj_doador', level=0, inplace=True)
doacoes_alckmin_nan.index.rename('nome_doador_receita', level=1, inplace=True)

doacoes_alckmin_nan

,,sum
cpf_cnpj_doador,nome_doador_receita,
60701190000104,ITAU UNIBANCO S.A.,4068000.00
04449030000130,ADOBE ASSESSORIA DE SERVICOS CADASTRAIS S.A.,2500000.00
61649810000168,SUCOCITRICO CUTRALE LTDA,2303000.00
51466860000156,SAO MARTINHO S/A,2272400.00
61186680000174,BANCO BMG SA,2225000.00
61442737000159,INDUSTRIAS BRASILEIRAS DE ARTIGOS REFRATARIOS - IBAR - LTDA,2190000.00
62011788000199,LITUCERA LIMPEZA E ENGENHARIA LTDA,2100000.00
10265949000177,COPERSUCAR S.A.,2010576.00
63935829000104,SPIRAL DO BRASIL LTDA,1771746.82


Após analisar os 3 DataFrames (PF, PJ e NaN) vemos que a maior parte das doações (tanto de empresas quanto de pessoas físicas) está no `doacoes_alckmin_nan`. Por conta disso, vamos seguir a análise utilizando esse DataFrame e deixar os outros de lado por enquanto.

In [18]:
doacoes_alckmin_nan_top20 = doacoes_alckmin_nan[0:20]

doacoes_alckmin_nan_top20 = doacoes_alckmin_nan_top20.reset_index()
#doacoes_alckmin_nan_top20 = doacoes_alckmin_nan_top20.set_index('cpf_cnpj_doador')

doacoes_alckmin_nan_top20

,cpf_cnpj_doador,nome_doador_receita,sum
0,60701190000104,ITAU UNIBANCO S.A.,4068000.00
1,04449030000130,ADOBE ASSESSORIA DE SERVICOS CADASTRAIS S.A.,2500000.00
2,61649810000168,SUCOCITRICO CUTRALE LTDA,2303000.00
3,51466860000156,SAO MARTINHO S/A,2272400.00
4,61186680000174,BANCO BMG SA,2225000.00
5,61442737000159,INDUSTRIAS BRASILEIRAS DE ARTIGOS REFRATARIOS ...,2190000.00
6,62011788000199,LITUCERA LIMPEZA E ENGENHARIA LTDA,2100000.00
7,10265949000177,COPERSUCAR S.A.,2010576.00
8,63935829000104,SPIRAL DO BRASIL LTDA,1771746.82
9,39318225000126,BRAZIL TRADING LTDA,1700000.00


In [19]:
query = text(
"""
    SELECT  cnpj_empresa,
            tipo_socio,
            cpf_cnpj_socio,
            qualificacao_socio,
            nome_socio
    FROM socios
    WHERE cnpj_empresa IN
    ('60701190000104',
    '4449030000130',
    '61649810000168',
    '51466860000156',
    '61186680000174',
    '61442737000159',
    '62011788000199',
    '10265949000177',
    '63935829000104',
    '39318225000126',
    '50220656000198',
    '19377221000101',
    '7359641000186',
    '33010851000174',
    '1637895000132',
    '2038394000100',
    '47080619001199',
    '5332851000155',
    '8411277000119',
    '48540421000131')
""")
engine = create_engine('sqlite:////mnt/hgfs/SharedVM/socios-brasil.sqlite')

socios = pd.read_sql(query, con=engine)

socios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 5 columns):
cnpj_empresa          177 non-null object
tipo_socio            177 non-null object
cpf_cnpj_socio        177 non-null object
qualificacao_socio    177 non-null object
nome_socio            177 non-null object
dtypes: object(5)
memory usage: 7.0+ KB


In [21]:
doacoes_alckmin_nan_top20 = doacoes_alckmin_nan_top20.merge(socios, how='inner', left_on='cpf_cnpj_doador',
                                right_on='cnpj_empresa')

doacoes_alckmin_nan_top20

,cpf_cnpj_doador,nome_doador_receita,sum,cnpj_empresa,tipo_socio,cpf_cnpj_socio,qualificacao_socio,nome_socio
0,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,JOAO ANTONIO DANTAS BEZERRA LEITE
1,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,CESAR PADOVAN
2,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,RICARDO VILLELA MARINO
3,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,CICERO MARCUS DE ARAUJO
4,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,CARLOS EDUARDO MONICO
5,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,CRISTIANE MAGALHAES TEIXEIRA PORTELLA
6,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,FERNANDO MARSELLA CHACON RUIZ
7,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,MARCO ANTONIO SUDANO
8,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,ANDREA MATTEUCCI PINOTTI
9,60701190000104,ITAU UNIBANCO S.A.,4068000.00,60701190000104,Pessoa Física,,Diretor,CARLOS HENRIQUE DONEGA AIDAR
